In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
from cellpose.io import imread
import glob
from pathlib import Path
from PIL import Image, ImageSequence
from tqdm import tqdm
import os
import os.path
from livecell_tracker import segment
from livecell_tracker import core
from livecell_tracker.core import datasets
from livecell_tracker.core.datasets import LiveCellImageDataset
from skimage import measure
from livecell_tracker.core import SingleCellTrajectory, SingleCellStatic
import detectron2
from detectron2.utils.logger import setup_logger

setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import cv2

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from livecell_tracker.segment.detectron_utils import gen_cfg

from livecell_tracker.segment.detectron_utils import (
    segment_detectron_wrapper,
    segment_images_by_detectron,
    convert_detectron_instance_pred_masks_to_binary_masks,
    convert_detectron_instances_to_label_masks,
)
from livecell_tracker.segment.detectron_utils import (
    convert_detectron_instance_pred_masks_to_binary_masks,
    convert_detectron_instances_to_label_masks,
    segment_images_by_detectron,
    segment_single_img_by_detectron_wrapper,
)


dataset_dir_path = Path(
    "../datasets/test_data_STAV-A549/DIC_data"
)

mask_dataset_path = Path("../datasets/test_data_STAV-A549/mask_data")

In [ ]:
mask_dataset = LiveCellImageDataset(mask_dataset_path, ext="png")
mask_dataset.time2url

In [ ]:
dic_dataset = LiveCellImageDataset(dataset_dir_path, ext="tif")

In [ ]:
dic_dataset.time2url

Convert label masks to single objects

In [ ]:
from skimage.measure import regionprops

single_cells = []

for time in mask_dataset.time2url:
    img = dic_dataset.get_img_by_time(time)
    seg_mask = mask_dataset.get_img_by_time(time)
    props_list = regionprops(seg_mask)
    for prop in props_list:
        single_cells.append(
            SingleCellStatic(
                timeframe=time,
                img_dataset = dic_dataset,
                mask_dataset = mask_dataset,
                bbox=prop.bbox,
                contour=prop.coords, # TODO: fix BUG here... should not be coords but contour
            )
        )


In [ ]:
# for testing
# single_cells = single_cells[:10]

In [ ]:
len(single_cells)

In [ ]:
single_cells_by_time = {}
for cell in single_cells:
    if cell.timeframe not in single_cells_by_time:
        single_cells_by_time[cell.timeframe] = []
    single_cells_by_time[cell.timeframe].append(cell)

In [ ]:
for time in single_cells_by_time:
    print(time, len(single_cells_by_time[time]))

Visualize one single cell

In [ ]:
sc = single_cells[0]

fig, axes = plt.subplots(1, 4, figsize=(10, 5))
sc.show(ax=axes[0])
sc.show_mask(ax=axes[1])
sc.show_contour_img(ax=axes[2])
sc.show_contour_mask(ax=axes[3])

Calculate overlap between two single cells

In [ ]:
sc.show_mask(padding=200)

In [ ]:
sc.show_contour_mask(padding=200)

In [ ]:
sc1 = single_cells_by_time[0][0]
sc2 = single_cells_by_time[0][1]

def compute_overlap_bf(sc1, sc2):
    # calculate overlap
    img_shape = sc1.get_img().shape
    # TODO: add a helper function in single_cell to returna mask with only the current cell in it.
    mask1 = sc1.get_contour_mask(padding=np.max(img_shape)).astype(bool)
    mask2 = sc2.get_contour_mask(padding=np.max(img_shape)).astype(bool)

    overlap_area = np.logical_and(mask1, mask2).sum()
    iou = overlap_area / (mask1 | mask2).sum()
    return overlap_area, iou

def bbox_overlap(bbox1, bbox2):
    # calculate overlap
    x1_min, y1_min, x1_max, y1_max = bbox1
    x2_min, y2_min, x2_max, y2_max = bbox2
    x_overlap = max(0, min(x1_max, x2_max) - max(x1_min, x2_min))
    y_overlap = max(0, min(y1_max, y2_max) - max(y1_min, y2_min))
    overlap_area = x_overlap * y_overlap
    bbox1_area = (x1_max - x1_min) * (y1_max - y1_min)
    bbox2_area = (x2_max - x2_min) * (y2_max - y2_min)
    iou = overlap_area / (bbox1_area + bbox2_area - overlap_area)
    return overlap_area, iou

def compute_overlap(sc1: SingleCellStatic, sc2: SingleCellStatic):
    bbox1, bbox2 = sc1.get_bbox(), sc2.get_bbox()
    bbox_overlap_area, bbox_iou = bbox_overlap(bbox1, bbox2)
    if bbox_iou <= 0:
        return 0, 0

    merged_bbox = (min(bbox1[0], bbox2[0]), min(bbox1[1], bbox2[1]), max(bbox1[2], bbox2[2]), max(bbox1[3], bbox2[3]))
    # calculate overlap
    # TODO: add a helper function in single_cell to returna mask with only the current cell in it.
    mask1 = sc1.get_contour_mask(crop=False)[merged_bbox[0]:merged_bbox[2], merged_bbox[1]:merged_bbox[3]]
    mask2 = sc2.get_contour_mask(crop=False)[merged_bbox[0]:merged_bbox[2], merged_bbox[1]:merged_bbox[3]]
    overlap_area = np.logical_and(mask1, mask2).sum()
    iou = overlap_area / (mask1 | mask2).sum()
    return overlap_area, iou
t1, t2 = 0, 1

In [ ]:
def test_compute_overlap():
    for sc in single_cells:
        for sc_tmp in single_cells:
            # compare two overlap algorithms
            overlap_area, iou = compute_overlap(sc, sc_tmp)
            overlap_area_bf, iou_bf = compute_overlap_bf(sc, sc_tmp)
            # print(overlap_area, overlap_area_bf)
            assert overlap_area == overlap_area_bf
            assert iou == iou_bf
# test_compute_overlap()

In [ ]:
import tqdm
def compute_overlaps(sc_list1, sc_list2):
    overlap_map = {}
    for sc1 in tqdm.tqdm(sc_list1, desc="Computing overlaps"):
        for sc2 in sc_list2:
            overlap_area, iou = compute_overlap(sc1, sc2)
            overlap_map[(sc1, sc2)] = (overlap_area, iou)
    # parallel version
    return overlap_map

overlap_map_by_time = {}
times = set(sorted(list(single_cells_by_time.keys())))
for time in times:
    if time + 1 not in times:
        print(f"Time {time} is the last time point, skipping")
        continue
    overlap_map_by_time[time] = compute_overlaps(single_cells_by_time[time], single_cells_by_time[time + 1])

In [ ]:
sc1, sc2 = single_cells_by_time[0][0], single_cells_by_time[1][0]
overlap_map_by_time[0][(sc1, sc2)]

In [ ]:
len(single_cells_by_time[2])

Check the cells visually

In [ ]:
# for time in overlap_map_by_time:
#     overlap_map = overlap_map_by_time[time]
#     for sc_tmp1, sc_tmp2 in overlap_map:
#         if sc_tmp1 == sc_tmp2:
#             continue
#         if overlap_map[(sc_tmp1, sc_tmp2)][0] > 0:
#             print(sc_tmp1.timeframe, sc_tmp2.timeframe, overlap_map[(sc_tmp1, sc_tmp2)])
#             fig, axes = plt.subplots(1, 6, figsize=(15, 5))
#             padding=50
#             sc_tmp1.show_contour_mask(crop=False, ax = axes[0])
#             sc_tmp2.show_contour_mask(crop=False, ax = axes[1])
#             sc_tmp1.show(crop=True, ax = axes[2], padding=padding)
#             sc_tmp2.show(crop=True, ax = axes[3], padding=padding)
#             sc_tmp1.show_contour_mask(crop=True, ax = axes[4], padding=padding)
#             sc_tmp2.show_contour_mask(crop=True, ax = axes[5], padding=padding)
#             plt.show()


In [ ]:
np.unique(sc1.mask_dataset.get_img_by_time(0))

In [ ]:
np.unique(sc1.mask_dataset.get_img_by_time(1))

```
sc.datasets["img"]
sc.datasets["mask"]
sc.datasets["label"]
sc.datasets["TRITC"]
```

In [ ]:
from livecell_tracker.segment.utils import match_mask_labels_by_iou
def match_mask_labels_by_iou(seg_label_mask, gt_label_mask, bg_label=0, return_all=False):
    """compute the similarity between ground truth mask and segmentation mask by intersection over union

    Parameters
    ----------
    seg_label_mask : _type_
        _description_
    gt_label_mask : _type_
        _description_
    bg_label : int, optional
        _description_, by default 0
    return_all : bool, optional
        _description_, by default False
    Returns
    -------
        A <gt2seg_map>, mapping ground truth keys to a dictionary of the best matching segmentation label and its iou
    """
    gt2seg_map = {}
    all_gt2seg_iou__map = {}
    # gets all the unique labels in the labeled_seg_mask and gtly_curated_mask
    seg_labels = np.unique(seg_label_mask)
    gt_labels = np.unique(gt_label_mask)

    temp_seg_mask = seg_label_mask.copy()
    temp_gt_mask = gt_label_mask.copy()

    for gt_label in gt_labels:
        if gt_label == bg_label:
            continue
        gt_label_key = gt_label
        all_gt2seg_iou__map[gt_label_key] = []
        gt2seg_map[gt_label_key] = {}
        temp_gt_mask = gt_label_mask.copy()
        # isolates the current cell in the temp gtly_curated_mask and gets its pixels to 1
        temp_gt_mask[temp_gt_mask != gt_label] = 0
        temp_gt_mask[temp_gt_mask != 0] = 1

        best_iou = 0
        for seg_label in seg_labels:
            if seg_label == bg_label:
                continue
            temp_seg_mask = seg_label_mask.copy()

            # isolate the current cell in the temp_seg_mask and set its pixels to 1
            temp_seg_mask[temp_seg_mask != seg_label] = 0
            temp_seg_mask[temp_seg_mask != 0] = 1

            matching_rows, matching_columns = np.where(temp_seg_mask == 1)
            intersection_area = (temp_gt_mask[matching_rows, matching_columns] == 1).sum()
            union_area = temp_gt_mask.sum() + temp_seg_mask.sum() - intersection_area
            iou = intersection_area / union_area
            io_gt = intersection_area / temp_gt_mask.sum()
            io_seg = intersection_area / temp_seg_mask.sum()
            all_gt2seg_iou__map[gt_label_key].append({
                "seg_label": seg_label,
                "iou": iou,
                "io_gt": io_gt,
                "io_seg": io_seg,
            })

            if iou > best_iou:
                best_iou = iou
                gt2seg_map[gt_label_key]["best_iou"] = iou
                gt2seg_map[gt_label_key]["seg_label"] = seg_label
    if return_all:
        return gt2seg_map, all_gt2seg_iou__map
    else:
        return gt2seg_map
    
match_mask_labels_by_iou(sc1.mask_dataset.get_img_by_time(2), sc1.mask_dataset.get_img_by_time(1), return_all=True)

## Apply correction CNN

In [ ]:
from livecell_tracker.model_zoo.segmentation.sc_correction import CorrectSegNet
ckpt = r"/home/ken67/LiveCellTracker-dev/notebooks/lightning_logs/version_real-02/checkpoints/epoch=3720-test_loss=0.0085.ckpt"

model = CorrectSegNet.load_from_checkpoint(ckpt)
model = model.cuda()
model = model.eval()

In [ ]:
single_cells_by_time[2][12].show_panel()

In [ ]:
single_cells_by_time[2][13].show_panel()

In [ ]:
from livecell_tracker.segment.ou_utils import create_ou_input_from_sc
from torchvision import transforms
from livecell_tracker.preprocess.utils import normalize_img_to_uint8
import torch
transforms = transforms.Compose(
        [
            transforms.Resize(size=(412, 412)),
        ]
)
from livecell_tracker.preprocess.utils import dilate_or_erode_mask
def create_ou_input_from_sc(sc: SingleCellStatic, padding_pixels: int = 0, dtype=float, remove_bg=True, one_object=True, scale=0):
    if remove_bg:
        img_crop = sc.get_contour_img(padding=padding_pixels).astype(dtype)
    else:
        img_crop = sc.get_img_crop(padding=padding_pixels).astype(dtype)
    img_crop = normalize_img_to_uint8(img_crop).astype(dtype)
    if one_object:
        sc_mask = sc.get_contour_mask(padding=padding_pixels)
        sc_mask = dilate_or_erode_mask(sc_mask.astype(np.uint8), scale_factor=scale).astype(bool)
        img_crop[~sc_mask] *= -1
    else:
        img_crop[sc.get_mask_crop(padding=padding_pixels) == 0] *= -1
    return img_crop


selected_sc_list = [single_cells_by_time[2][12], single_cells_by_time[2][13]]
for i, sc in enumerate(selected_sc_list):
    print("sc index:", i)
    ou_input = create_ou_input_from_sc(sc, padding_pixels=100, remove_bg=False, scale=0.8)
    ou_input = transforms(torch.tensor([ou_input]))
    ou_input = torch.stack([ou_input, ou_input, ou_input], dim=1)
    # ou_input = ou_input.permute(0, 2, 3, 1)
    ou_input = ou_input.float().cuda()
    output = model(ou_input)

    # visualize the input and all 3 output channels
    fig, axes = plt.subplots(1, 5, figsize=(15, 5))
    axes[0].imshow(ou_input[0, 0].cpu().detach().numpy())
    axes[0].set_title("input")
    axes[1].imshow(output[0, 0].cpu().detach().numpy())
    axes[1].set_title("output c0")
    axes[2].imshow(output[0, 1].cpu().detach().numpy())
    axes[2].set_title("output c1")
    axes[3].imshow(output[0, 2].cpu().detach().numpy())
    axes[3].set_title("output c2")
    axes[4].imshow(sc.get_mask_crop(padding=50, dtype=int))
    axes[4].set_title("original mask")
    plt.show()

[TODO] judge if a case is oversegmentation
inputs:
    an input mask with other cells included
    a corected label mask
    


In [ ]:
from livecell_tracker.segment.utils import compute_match_label_map

t1, t2, label_map = compute_match_label_map(1, 2, mask_dataset)

In [ ]:
label_map